<a href="https://colab.research.google.com/github/sassgabai/pyspark-projects/blob/main/web_scrapping_aws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e0267dd00fc32d44171a04b674a641e7d019b46ec1796246fd73d7970d29004d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [31]:
from bs4 import BeautifulSoup
import requests
from pyspark.sql import SparkSession

In [85]:
spark = SparkSession.builder.getOrCreate()

In [32]:
URL = 'https://www.amazon.com/s?k=dragon+ball+z&language=en_US&crid=221T1AZ8FNVV9&sprefix=dragon+ball+%2Caps%2C235&ref=nb_sb_noss_2'

In [33]:
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

In [34]:
data = requests.get(URL, headers=HEADERS)

In [35]:
soup = BeautifulSoup(data.content, 'html.parser')

In [38]:
get_links = soup.find_all('a', attrs={'class':'a-link-normal s-no-outline'})

In [51]:
links_list =[]

for link in get_links:
  link_url = f'https://amazon.com{link.get("href")}'
  links_list.append(link_url)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [99]:
data_schema = ['description','price','title']
data_collected = []

In [100]:
for link in links_list[:5]:
  link_data = requests.get(link, headers=HEADERS)
  link_soup = BeautifulSoup(link_data.content, 'html.parser')

  get_header = link_soup.find('span', attrs={'class':'a-size-large product-title-word-break'}).text.strip()
  get_price = link_soup.find('div', attrs={'class':'a-section a-spacing-micro'})\
                       .find('span', attrs={'class':'a-offscreen'}).text.strip().replace('$','')


  get_description = link_soup.find('div', attrs={'class':'a-section a-spacing-medium a-spacing-top-small'})\
                             .find('ul', attrs={'class':'a-unordered-list a-vertical a-spacing-mini'})\

  for desc in get_description:
    list_desc = [desc.text.strip() for desc in get_description if desc.text.strip()]

  data_collected.append({
      'title': get_header,
      'price': get_price,
      'description': list_desc
  })



In [101]:
df = spark.createDataFrame(data_collected, data_schema)\
          .select('title','description','price')

In [103]:
df.show(100, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|title                    

In [15]:
soup = BeautifulSoup(data.content, 'html.parser')

In [17]:
links = soup.find_all('a', attrs={'class':'a-link-normal s-no-outline'})

In [23]:
web_href = links[0].get('href')

In [26]:
product_link = f'https://amazon.com{web_href}'

In [27]:
product_link

'https://amazon.com/sspa/click?ie=UTF8&spc=MTo2MjM2OTcwMjc2NDg5MDcyOjE3MTAzNTczMzM6c3BfYXRmOjIwMDA1ODU4NjkwNjk3MTo6MDo6&url=%2FDragon-Action-Figure-Supplies-Collectible%2Fdp%2FB08RXXT4Y9%2Fref%3Dsr_1_1_sspa%3Fcrid%3D221T1AZ8FNVV9%26dib%3DeyJ2IjoiMSJ9.04ZTNmlepbeMTvAUp8d5obTSeGzTQkk_BkWOZ4UCerP0lf3SLtAq5SGFavMAAKlZ9mN8boEieBaepp9gXRH9NBH0Dc0ZaqhjzwdaBPku3OFzeLKbtMVDiaYevayMyl8EAmF_ST_lwPlqmlseDS5tfaDaiXIDc1tv5SpZ4jaI8DYWvKvgX37JoY83RJ8hArFD6WNwsagVRPcMwb8vyaeb3giv40uaaIFWEwDuu-if7EsqftGTf8XRwX8bDQlhha85MpQjUCgCMh6jOux0nMeUc0gcrkUMeUyxbcllEDF48fA.fML1qZtOYBmHlKPj2kIMe8yNUN6oa7fls-YchCFraGY%26dib_tag%3Dse%26keywords%3Ddragon%2Bball%2Bz%26qid%3D1710357333%26sprefix%3Ddragon%2Bball%2B%252Caps%252C235%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1'

In [28]:
product_webpage = requests.get(product_link, headers=HEADERS)

In [29]:
product_webpage

<Response [200]>